In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

Tuning the hyperparameters 

In [ ]:
### Hyperparameters, change accordingly
Channel_uses = 4
Epochs = 2500
Noise_variance = 1e-4
Pert_variance = 1e-4
Batch_size = 1024
# init_losses_vec = np.ones(128)

Transmission loss

In [ ]:
#tramsmitter loss function 
def tx_loss(y_true, y_pred): 
#     loss = - y_true*keras.backend.log(y_pred)

    return -y_true*y_pred

PERTURBATION

In [ ]:
#pertubation
def perturbation(d):
    W = tf.keras.backend.random_normal(shape = (2*Channel_uses,),
    mean=0.0,stddev=Pert_variance**0.5,dtype=None,seed=None)
    d = ((1-Pert_variance)**0.5)*d + W
    return d

TRANSMISSON  MODEL

In [ ]:
def Int_layer(y):
    w = y[:,y.shape[-1]//2:] - y[:,:y.shape[-1]//2]
    print(w.shape)
    t = -keras.backend.sum(w*w)
#     t = keras.backend.exp(-t/Pert_variance**2)/(np.pi*Pert_variance**2)**Channel_uses
    return t

In [ ]:
# tx layers
tx_input = keras.layers.Input((1,), name='tx_input')
x = keras.layers.BatchNormalization()(tx_input)
x = keras.layers.Dense(units=10*Channel_uses, activation='elu', name='tx_10')(x)
x = keras.layers.Dense(units=2*Channel_uses, activation='elu', name='tx_out')(x)
xp = keras.layers.Lambda(perturbation, name='Xp')(x)
concat = keras.layers.concatenate([x,xp], axis=1)
policy = keras.layers.Lambda(Int_layer)(concat)
print(concat.shape)

(None, 8)
(None, 16)


In [ ]:
tx_model = keras.models.Model(inputs=tx_input, outputs=concat)

In [ ]:
tx_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 tx_input (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 1)           4           ['tx_input[0][0]']               
 alization)                                                                                       
                                                                                                  
 tx_10 (Dense)                  (None, 40)           80          ['batch_normalization[0][0]']    
                                                                                                  
 tx_out (Dense)                 (None, 8)            328         ['tx_10[0][0]']              

In [ ]:
pert_model = keras.models.Model(inputs=tx_input, outputs=policy)

In [ ]:
pert_model.compile(loss=tx_loss, optimizer='sgd')
pert_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 tx_input (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 1)           4           ['tx_input[0][0]']               
 alization)                                                                                       
                                                                                                  
 tx_10 (Dense)                  (None, 40)           80          ['batch_normalization[0][0]']    
                                                                                                  
 tx_out (Dense)                 (None, 8)            328         ['tx_10[0][0]']            

RECIEVER MODEL

In [ ]:
rx_input = keras.layers.Input((2*Channel_uses,), name='rx_input')
# channel layer
y = keras.layers.Lambda(lambda x: x+keras.backend.random_normal(
        shape = (2*Channel_uses,), mean=0.0, stddev=Noise_variance**0.5), name='channel')(rx_input)

y = keras.layers.Dense(2*Channel_uses, activation='relu', name='rx_2')(y)
y = keras.layers.Dense(10*Channel_uses, activation='relu', name='rx_10')(y)
pred = keras.layers.Dense(1, activation='relu', name='rx_output')(y)

In [ ]:
rx_model = keras.models.Model(inputs=rx_input, outputs=pred)
rx_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rx_input (InputLayer)       [(None, 8)]               0         
                                                                 
 channel (Lambda)            (None, 8)                 0         
                                                                 
 rx_2 (Dense)                (None, 8)                 72        
                                                                 
 rx_10 (Dense)               (None, 40)                360       
                                                                 
 rx_output (Dense)           (None, 1)                 41        
                                                                 
Total params: 473
Trainable params: 473
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rx_model.compile(loss='mse', optimizer='sgd')

In [ ]:
data_numbers = np.random.uniform(0,1,(Batch_size,))
y = tx_model.predict(data_numbers)
print(y.shape)
XP = y[:,y.shape[-1]//2:]
estimated_vector  = np.squeeze(rx_model.predict(XP))
print(estimated_vector.shape, data_numbers.shape)

32/32 [==============================] - 0s 2ms/step
(1024, 16)
32/32 [==============================] - 0s 3ms/step
(1024,) (1024,)


In [ ]:
l = (estimated_vector-data_numbers)**2
l_hat = rx_model.predict(tx_model.predict(data_numbers)[:,2*Channel_uses:])

32/32 [==============================] - 0s 6ms/step


MODEL TRAINING 

In [ ]:
for i in range(Epochs):
    data_numbers = np.random.uniform(0,1,(Batch_size,))
    y = tx_model.predict(data_numbers)
    XP = y[:,y.shape[-1]//2:]
    estimated_vector= np.squeeze(rx_model.predict(XP))
    l = (estimated_vector-data_numbers)**2
    l_hat = rx_model.predict(tx_model.predict(data_numbers)[:,2*Channel_uses:])
    pert_model.fit(data_numbers, l_hat, batch_size=Batch_size, epochs=1, verbose=0)
#     print("Tx-done")
    rx_model.fit(XP, data_numbers, batch_size=Batch_size, epochs=1)
#     print("Rx-done")

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 13ms/step - loss: 3.5653e-04


PREDICTION

In [ ]:
data_numbers = np.random.uniform(0,1,(10,))
y = tx_model.predict(data_numbers)
XP = y[:,y.shape[-1]//2:]
estimated_vector= np.squeeze(rx_model.predict(XP))
print(data_numbers)
print(estimated_vector)

1/1 [==============================] - 0s 66ms/step
[0.02186184 0.15256673 0.95721879 0.03973548 0.58025894 0.45049921
 0.1736766  0.58121911 0.141529   0.36253852]
[0.08061925 0.1505917  0.94963676 0.08454078 0.6027512  0.46671057
 0.17152888 0.603702   0.13992608 0.37395108]


In [ ]:
# Generate 8-PAM data
modulation_data = np.linspace(0,1,8)
modulation_inds = np.linspace(0,7,8)
print(modulation_data)
modulation_inds = modulation_inds.astype(int)
print(modulation_inds)
decision_regions = []
for i in range(len(modulation_data) -1):
  decision_regions.append((modulation_data[i]+modulation_data[i+1])/2.0)

print(decision_regions)

[0.         0.14285714 0.28571429 0.42857143 0.57142857 0.71428571
 0.85714286 1.        ]
[0 1 2 3 4 5 6 7]
[0.07142857142857142, 0.21428571428571427, 0.3571428571428571, 0.5, 0.6428571428571428, 0.7857142857142856, 0.9285714285714286]


In [ ]:
y = tx_model.predict(modulation_data)
XP = y[:,y.shape[-1]//2:]
estimated_vector= np.squeeze(rx_model.predict(XP))
print(modulation_data)
print(estimated_vector)

est_sig = []
for i in estimated_vector:
  index = (np.abs(modulation_data-i)).argmin()
  est_sig.append(modulation_inds[index])
  
est_sig = np.array(est_sig)
print(est_sig)

1/1 [==============================] - 0s 64ms/step
[0.         0.14285714 0.28571429 0.42857143 0.57142857 0.71428571
 0.85714286 1.        ]
[0.07667553 0.12409949 0.276259   0.4316259  0.5823426  0.7200975
 0.8503865  0.97460437]
[1 1 2 3 4 5 6 7]


ACCURACY SCORE 

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(est_sig, modulation_inds))

0.875
